**Ultralytics download**
to use TOLOv11 Model

In [ ]:
%pip install ultralytics
%pip install opencv-python
%pip install scikit-learn

In [ ]:
%pip install --upgrade sympy

In [1]:
from ultralytics import YOLO

# Load a COCO-pretrained YOLOv8n model
model = YOLO("yolov8n.pt")

# Display model information (optional)
model.info()


YOLOv8n summary: 225 layers, 3,157,200 parameters, 0 gradients, 8.9 GFLOPs


(225, 3157200, 0, 8.8575488)

In [ ]:
# Train the model
results = model.train(data="coco8.yaml", epochs=100, imgsz=640)

In [ ]:
from ultralytics import YOLO

# Load a COCO-pretrained YOLOv8n model
model = YOLO("yolov8n.pt")

# Display model information (optional)
model.info()

# Train the model with a validation set split
results = model.train(data="coco8.yaml", epochs=100, imgsz=640, val=0.2) # val=0.2 splits 20% of the data for validation


In [3]:
import cv2
# Initialize the webcam
cap = cv2.VideoCapture(0)  # 0 for default webcam

while cap.isOpened():
    # Read a frame from the webcam
    success, frame = cap.read()

    if success:
        # Run inference on the frame
        results = model(frame)

        # Get the annotated frame with bounding boxes
        annotated_frame = results[0].plot()

        # Display the annotated frame
        cv2.imshow("YOLOv8 Object Detection", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        print("Error reading frame from webcam.")
        break

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()

